# Lecture 5 - Unit testing

## 📕 Today's Agenda
---
 * [Unittest module](#Unittest-module)
 * Simple unit test
 * Executing tests
 * Setup and teardown
 * DDT
 * Mocking
 * Patching
 * Skipping tests and errors

## 🧪 Theory
---

### Unittest module

There are many modules for testing Python code, one on we will focus on is `unittest`. Other well known module for testing is `pytest`.
Unittest is a entire framework for testing your Py code, you can make tests, test suites and run tests with built-in runner.


### Simple unit test

Let's build a simple test for a simple function.

In [5]:
from math import pi
import unittest

def circle_area(radius):
    if not isinstance(radius, float) or not isinstance(radius, int):
        raise TypeError('Radius must be integer of float.')
    if radius <= 0:
        raise ValueError('Radius must be greater then zero.')
    return pi * radius ** 2

class TestMyFunctions(unittest.TestCase):
    def test_circle_area(self):
        with self.assertRaises(TypeError):
            circle_area(None)
        with self.assertRaises(ValueError):
            circle_area(-1)
        self.assertEqual(circle_area(1), 0.7853981634)

First steps to consider when creating tests:
1. Create a class that extends `unittest.TestCase'. Name this class prefixed with 'Test'.
2. Create functions in this class for testing functions. Each test function name must be prefixed with 'test_'.
3. Write test code in test functions.

NOTE: Prefixing classes and functions names will help test framework to auto-discover test.

**Run tests**

To run a python file with tests, place this snippet at the bottom.
```python
if __name__ == '__main__':
    unittest.main()
```

## 👩‍💻 Practice
---

## 🏠 Homework
---